In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from nltk import word_tokenize,sent_tokenize
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from collections import OrderedDict
import pprint

In [2]:
pd.read_csv('People with title final.csv')

,Unnamed: 0,name,male,culture,house,book1,book2,book3,book4,book5,isMarried,isNoble,age,numDeadRelations,boolDeadRelations,popularity,identity,isAlive
0,0,Viserys II Targaryen,1,NaN,NaN,0,0,0,0,0,0,0,NaN,11,1,0.605351,king,0
1,1,Walder Frey,1,Rivermen,House Frey,1,1,1,1,1,1,1,97.0,1,1,0.896321,knight,1
2,2,Addison Hill,1,NaN,House Swyft,0,0,0,1,0,0,1,NaN,0,0,0.267559,lord,1
3,3,Aemma Arryn,0,NaN,House Arryn,0,0,0,0,0,1,1,23.0,0,0,0.183946,official,0
4,4,Sylva Santagar,0,Dornish,House Santagar,0,0,0,1,0,1,1,29.0,0,0,0.043478,noble,1
5,5,Tommen Baratheon,1,NaN,NaN,0,0,0,0,0,0,0,NaN,5,1,1.000000,king,1
6,6,Valarr Targaryen,1,Valyrian,House Targaryen,0,0,0,0,0,1,1,26.0,0,0,0.431438,prince,0
7,7,Viserys I Targaryen,1,NaN,NaN,0,0,0,0,0,0,0,NaN,5,1,0.678930,king,0
8,8,Wilbert,1,NaN,NaN,0,0,1,0,0,0,1,NaN,0,0,0.006689,knight,0
9,9,Wilbert Osgrey,1,NaN,House Osgrey,0,0,0,0,0,0,1,NaN,0,0,0.020067,knight,1


In [9]:
def get_sentiment(name):
    response=requests.get("http://awoiaf.westeros.org/index.php/"+ name.replace(' ','_'))
    results_page=BeautifulSoup(response.content,'lxml')
    # paragraph = results_page.find('div',class_='mw-parser-output').find('p').get_text()
    para = results_page.find_all('p')
    para_list = list()
    for i in para:
        para_list.append(i.get_text())
    summary_sentences = []
    candidate_sentences = {}
    candidate_sentence_counts = {}
    text = r'{}'.format(para_list)
    # print(text.raw())
    striptext = text.replace('\n\n', ' ')
    striptext = striptext.replace('\n', ' ')
    striptext = striptext.replace('\\n', ' ')
    striptext = striptext.replace('[]', ' ')
    words = word_tokenize(striptext)
    lowercase_words = [word.lower() for word in words
                      if word not in stopwords.words() and word.isalpha()] 
    
    def get_pos_neg_words():    
        def get_words(url):
            import requests
            words = requests.get(url).content.decode('latin-1')
            word_list = words.split('\n')
            index = 0
            while index < len(word_list):
                word = word_list[index]
                if ';' in word or not word:
                    word_list.pop(index)
                else:
                    index+=1
            return word_list

        #Get lists of positive and negative words
        p_url = 'http://ptrckprry.com/course/ssd/data/positive-words.txt'
        n_url = 'http://ptrckprry.com/course/ssd/data/negative-words.txt'
        positive_words = get_words(p_url)
        negative_words = get_words(n_url)
        return positive_words,negative_words
    
    positive_words,negative_words = get_pos_neg_words()
    
    def do_pos_neg_sentiment_analysis(text,debug=False):
        positive_words,negative_words = get_pos_neg_words()
        from nltk import word_tokenize
        results = list()
        cpos = cneg = lpos = lneg = 0
        for word in text:
            if word in positive_words:
                if debug:
                    print("Positive",word)
                cpos+=1
            if word in negative_words:
                if debug:
                    print("Negative",word)
                cneg+=1
        results.append((cpos/len(text),cneg/len(text)))
        return results
    pos = do_pos_neg_sentiment_analysis(lowercase_words,debug=False)[0][0]
    neg = do_pos_neg_sentiment_analysis(lowercase_words,debug=False)[0][1]
    return f'positive rate: {pos} negative rate: {neg}'

In [10]:
get_sentiment('Viserys II Targaryen')

'positive rate: 0.046875 negative rate: 0.04403409090909091'